<a href="https://colab.research.google.com/github/EvagAIML/014-NLP-Model-v1/blob/main/Medical_Diagnostic_Tool_v12.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Executive Summary**
This notebook demonstrates a **Medical Diagnostic Tool** using **RAG (Retrieval-Augmented Generation)**.
It uses the **Merck Manual** as a knowledge base and **Mistral-7B** as the LLM.
Key features:
1.  **RAG Pipeline**: Retrieves relevant medical context with page citations.
2.  **Robust Evaluation**: Uses **LLM-as-a-Judge** (Mistral-7B) to score responses on a **1-5 scale** for **Groundedness** and **Relevance**.
3.  **Strict Scoring**: Prompts enforce a strict `SCORE: X` format to ensure reliable numerical parsing.
4.  **Professional Tone**: Adheres to strict medical persona constraints.



## **Medical Diagnostic Assistant: AI-Powered Question Answering**

## Problem Statement

### Business Context
The healthcare industry is rapidly evolving, with professionals facing increasing challenges in managing vast volumes of medical data while delivering accurate and timely diagnoses. The need for quick access to comprehensive, reliable, and up-to-date medical knowledge is critical for improving patient outcomes and ensuring informed decision-making in a fast-paced environment.

Healthcare professionals often encounter information overload, struggling to sift through extensive research and data to create accurate diagnoses and treatment plans. This challenge is amplified by the need for efficiency, particularly in emergencies, where time-sensitive decisions are vital. Furthermore, access to trusted, current medical information from renowned manuals and research papers is essential for maintaining high standards of care.

### Objective
As an AI specialist, the task is to develop a **Retrieval-Augmented Generation (RAG)** solution using renowned medical manuals to address healthcare challenges. The objective is to **understand** issues like information overload, **apply** AI techniques to streamline decision-making, **analyze** its impact on diagnostics and patient outcomes, **evaluate** its potential to standardize care practices, and **create** a functional prototype demonstrating its feasibility and effectiveness.

**Common Questions to Answer**

1. **Diagnostic Assistance**: "What are the common symptoms and treatments for pulmonary embolism?"
2. **Surgical Inquiry**: "What are the common symptoms for appendicitis, and can it be cured via medicine? If not, what surgical procedure should be followed to treat it?"
3. **Drug Information**: "Can you provide the trade names of medications used for treating hypertension?"
4. **Treatment Plans**: "What are the first-line options and alternatives for managing rheumatoid arthritis?"
5. **Dermatology**: "What are the effective treatments or solutions for addressing sudden patchy hair loss, commonly seen as localized bald spots on the scalp, and what could be the possible causes behind it?"
6. **Neurology**: "What treatments are recommended for a person who has sustained a physical injury to brain tissue, resulting in temporary or permanent impairment of brain function?"
7. **Emergency Care**: "What are the necessary precautions and treatment steps for a person who has fractured their leg during a hiking trip, and what should be considered for their care and recovery?"
8. **Specialty Knowledge**: "What are the diagnostic steps for suspected endocrine disorders?"
9. **Critical Care Protocols**: "What is the protocol for managing sepsis in a critical care unit?"

### Data Description
The **Merck Manuals** are medical references published by the American pharmaceutical company Merck & Co., that cover a wide range of medical topics, including disorders, tests, diagnoses, and drugs.

The manual is provided as a PDF: `014-NLP-PROJ-medical_diagnosis_manual_19.pdf`

### 1. Setup and Initialization\n\n**Outcome**: Environment configured with necessary libraries including LangChain, ChromaDB, and llama-cpp-python.\n\nAll dependencies are installed to support GPU-accelerated inference and vector operations.

In [ ]:
# Installation for GPU llama-cpp-python
!CMAKE_ARGS="-DLLAMA_CUBLAS=on" FORCE_CMAKE=1 pip install llama-cpp-python==0.2.28 --force-reinstall --no-cache-dir -q 2>/dev/null

# Installing other dependencies
!pip install huggingface_hub==0.35.3 pandas==2.2.2 tiktoken==0.12.0 pymupdf==1.26.5 langchain==0.3.27 langchain-community==0.3.31 chromadb==1.1.1 sentence-transformers==5.1.1 numpy==1.26.4 -q 2>/dev/null

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 83.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 282.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 321.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 310.3 MB/s eta 0:00:00


**Note:** Please restart the runtime after installation.

In [ ]:
# ============================================================\n# Data Loading & Chunking\n# ============================================================\n# ============================================================\n# Imports and Configuration\n# ============================================================\nimport os
import pandas as pd
from huggingface_hub import hf_hub_download
from llama_cpp import Llama
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_community.embeddings.sentence_transformer import SentenceTransformerEmbeddings
from langchain_community.vectorstores import Chroma

# Define the questions globally
questions = [
    "What are the common symptoms and treatments for pulmonary embolism?",
    "What are the common symptoms for appendicitis, and can it be cured via medicine? If not, what surgical procedure should be followed to treat it?",
    "Can you provide the trade names of medications used for treating hypertension?",
    "What are the first-line options and alternatives for managing rheumatoid arthritis?",
    "What are the effective treatments or solutions for addressing sudden patchy hair loss, commonly seen as localized bald spots on the scalp, and what could be the possible causes behind it?",
    "What treatments are recommended for a person who has sustained a physical injury to brain tissue, resulting in temporary or permanent impairment of brain function?",
    "What are the necessary precautions and treatment steps for a person who has fractured their leg during a hiking trip, and what should be considered for their care and recovery?",
    "What are the diagnostic steps for suspected endocrine disorders?",
    "What is the protocol for managing sepsis in a critical care unit?"
]

### 2. Question Answering using LLM (Vanilla)\n\n**Outcome**: Baseline performance established using off-the-shelf Mistral-7B model.\n\nThe model is tested on standard medical queries without external context to identify hallucinations and knowledge gaps.

In [ ]:
# Load LLM
model_name_or_path = "TheBloke/Mistral-7B-Instruct-v0.2-GGUF"
model_basename = "mistral-7b-instruct-v0.2.Q4_K_M.gguf"

model_path = hf_hub_download(
    repo_id=model_name_or_path,
    filename=model_basename
)

# Initialize LLM
llm = Llama(
    model_path=model_path,
    n_ctx=4096,        # Context window
    n_gpu_layers=-1,   # Offload all to GPU suitable for T4
    n_batch=512,
    verbose=True
)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


mistral-7b-instruct-v0.2.Q4_K_M.gguf:   0%|          | 0.00/4.37G [00:00<?, ?B/s]

AVX = 1 | AVX_VNNI = 0 | AVX2 = 1 | AVX512 = 1 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 1 | SSE3 = 1 | SSSE3 = 1 | VSX = 0 | 


In [ ]:
def generate_response(query, max_tokens=2048, temperature=0.1, top_p=0.95, top_k=50, system_prompt=None):
    if system_prompt:
        prompt = f"[INST] {system_prompt}\n{query} [/INST]"
    else:
        prompt = f"[INST] {query} [/INST]"

    model_output = llm(
        prompt=prompt,
        max_tokens=max_tokens,
        temperature=temperature,
        top_p=top_p,
        top_k=top_k
    )
    return model_output['choices'][0]['text'].strip()

vanilla_results = []
for i, q in enumerate(questions, 1):
    print(f"--- Question {i} ---")
    print(f"Q: {q}")
    ans = generate_response(q)
    print(f"A: {ans}\n")
    vanilla_results.append(ans)

--- Question 1 ---
Q: What are the common symptoms and treatments for pulmonary embolism?
A: A pulmonary embolism (PE) is a serious condition that occurs when a blood clot or other foreign substance travels to the lungs and blocks one or more of the arteries in the lungs. This can lead to decreased blood flow, oxygen levels in the body, and potentially life-threatening complications. Here are some common symptoms of PE:

1. Shortness of breath: This is often the first symptom of a pulmonary embolism. It may feel like you're unable to get enough air or that you're breathing very hard for no reason.
2. Chest pain: PE can cause sharp, stabbing chest pain, especially when taking deep breaths or coughing. The pain may worsen with movement or physical activity.
3. Rapid heart rate: A fast heartbeat is a common symptom of PE. Your heart may beat quickly and irregularly.
4. Coughing up blood: In some cases, PE can cause you to cough up blood or bloody mucus.
5. Swelling: PE can cause swelling 

Llama.generate: prefix-match hit


A: Appendicitis is a medical condition characterized by inflammation of the appendix, a small tube-shaped organ located in the lower right side of the abdomen. The following are common symptoms of appendicitis:

1. Abdominal pain: The pain typically starts as a mild discomfort around the navel and then moves to the lower right side of the abdomen. The pain may be constant or intermittent, and it may worsen with movement or pressure on the abdomen.
2. Loss of appetite: People with appendicitis often lose their appetite due to abdominal discomfort.
3. Nausea and vomiting: Vomiting is a common symptom of appendicitis, especially if the inflammation is severe.
4. Fever: A fever may be present in some cases of appendicitis.
5. Diarrhea or constipation: Some people with appendicitis experience diarrhea, while others have constipation.
6. Abdominal swelling: The abdomen may become swollen and tender to the touch.
7. Inability to pass gas: Passing gas can be difficult due to the pressure on th

Llama.generate: prefix-match hit


A: Yes, I can provide a list of some common trade names of medications used for treating hypertension (high blood pressure). Please note that this is not an exhaustive list and there are many other brand names for various classes of antihypertensive drugs. Here are some examples:

1. Thiazide diuretics: Diuril (chlorothiazide), Maxzide (triamterene/hydrochlorothiazide), Dyazide (hydrochlorothiazide/amloride)
2. Beta-blockers: Tenormin (atenolol), Lopressor (metoprolol), Toprol XL (metoprolol succinate), Bisoprolol, Labetalol
3. ACE inhibitors: Zestril (lisinopril), Monoprast (quanapril), Vasotec (enalapril), Lotensin (benazepril)
4. ARBs (Angiotensin Receptor Blockers): Cozaar (losartan), Diovan (valsartan), Avapro (irbesartan), Micardis (telmisartan)
5. Calcium channel blockers: Norvasc (amlodipine), Procardia XL (nifedipine extended-release), Diltia XT (diltiazem extended-release)
6. Diuretics - Potassium-sparing: Aldactone (spironolactone), Microzide (hydrochlorothiazide/triamterene

...

A: If a person has fractured their leg during a hiking trip, it is important that they receive proper medical attention as soon as possible. Here are some necessary precautions and treatment steps:

1. Assess the situation: Check the severity of the injury and assess the overall condition of the person. If the fracture is open or the person is in severe pain, shock, or unable to move, do not attempt to move them without professional help. Call for emergency medical services if necessary.

2. Provide first aid: If the person is stable and able to be moved, provide first aid by immobilizing the leg with a splint or sling. Use any available materials such as sticks, branches, or clothing to create a makeshift splint. Be careful not to move the leg excessively or apply too much pressure that could worsen the injury.

3. Keep the person warm and comfortable: Provide shelter from the elements and keep the person warm with blankets or extra clothing. Offer water or other fluids to help preven

Llama.generate: prefix-match hit


A: 1. Thorough Clinical Evaluation: A comprehensive clinical evaluation is essential to identify any symptoms suggestive of an endocrine disorder. This may include assessing medical history, family history, and performing a physical examination.

2. Laboratory Tests: Various laboratory tests are used to diagnose endocrine disorders. These tests may include:
   - Hormone level measurements: Measuring the levels of specific hormones in the blood or urine can help identify an endocrine disorder. For example, measuring insulin levels for diabetes mellitus or thyroid function tests for thyroid disorders.
   - Imaging studies: Imaging techniques such as X-rays, CT scans, MRI, and ultrasound may be used to visualize glands and organs related to endocrine function.
   - Genetic testing: In some cases, genetic testing may be necessary to identify specific gene mutations associated with certain endocrine disorders.

3. Diagnostic Procedures: Several diagnostic procedures can help confirm a suspe

Llama.generate: prefix-match hit


A: Sepsis is a life-threatening condition caused by the body's response to an infection, and managing it effectively in a critical care unit requires a multidisciplinary approach. Here are some key steps in the protocol for managing sepsis in a critical care unit:

1. Early recognition: Recognizing sepsis early is crucial for effective management. Look out for signs and symptoms such as fever, chills, tachycardia, tachypnea, altered mental status, lactate elevation, and low urine output. Use the Sequential Organ Failure Assessment (SOFA) score or Quick Sequential [Sepsis-related] Organ Failure Assessment (qSOFA) to identify patients at risk of sepsis.

2. Resuscitation: Begin resuscitation as soon as possible, focusing on restoring intravascular volume and maintaining adequate tissue perfusion. Administer intravenous fluids based on the patient's hemodynamic status and urine output. Consider using vasopressors to maintain mean arterial pressure (MAP) ≥65 mmHg if necessary.

3. Antibiot

In [ ]:
def generate_response(query, max_tokens=2048, temperature=0.1, top_p=0.95, top_k=50, system_prompt=None):
    if system_prompt:
        prompt = f"[INST] {system_prompt}\n{query} [/INST]"
    else:
        prompt = f"[INST] {query} [/INST]"

    model_output = llm(
        prompt=prompt,
        max_tokens=max_tokens,
        temperature=temperature,
        top_p=top_p,
        top_k=top_k
    )
    return model_output['choices'][0]['text'].strip()

### Baseline Answers (Vanilla)

In [ ]:
vanilla_results = []
for i, q in enumerate(questions, 1):
    print(f"--- Question {i} ---")
    print(f"Q: {q}")
    ans = generate_response(q)
    print(f"A: {ans}\n")
    vanilla_results.append(ans)

--- Question 1 ---
Q: What are the common symptoms and treatments for pulmonary embolism?


Llama.generate: prefix-match hit


A: A pulmonary embolism (PE) is a serious condition that occurs when a blood clot or other foreign substance travels to the lungs and blocks one or more of the arteries supplying the lungs with blood. This can lead to decreased oxygen supply to the body, which can be life-threatening. Here are some common symptoms of pulmonary embolism:

1. Shortness of breath: This is often the first and most prominent symptom of PE. The person may feel like they're unable to get enough air or that they're breathing too hard for the level of activity they're engaged in.
2. Chest pain: PE can cause sharp, stabbing chest pain that worsens with deep breaths or physical activity. The pain may be described as feeling like a heavy weight on the chest.
3. Rapid heart rate: PE can cause the heart to beat faster than normal in an attempt to pump more blood through the body and deliver oxygen to the lungs.
4. Coughing up blood: In some cases, PE can cause coughing up blood or bloody sputum.
5. Swelling: PE can 

Llama.generate: prefix-match hit


A: Appendicitis is a medical condition characterized by inflammation of the appendix, a small tube-shaped organ located in the lower right side of the abdomen. The following are common symptoms of appendicitis:

1. Abdominal pain: The pain typically starts as a mild discomfort around the navel that may come and go. However, it eventually localizes to the lower right quadrant of the abdomen and becomes constant, severe, and sharp in nature.
2. Loss of appetite: People with appendicitis often lose their appetite due to the pain and discomfort.
3. Nausea and vomiting: Vomiting is a common symptom of appendicitis, especially when the inflammation is severe.
4. Fever: A fever above 100.4°F (38°C) may be present in some cases.
5. Diarrhea or constipation: Appendicitis can cause both diarrhea and constipation, depending on the location of the inflammation and the pressure it puts on the intestines.
6. Abdominal swelling: The affected area may become tender, swollen, and rigid to touch.
7. Ina

...

A: If a person has fractured their leg during a hiking trip, it is important that they receive proper medical attention as soon as possible. Here are some necessary precautions and treatment steps:

1. Assess the situation: Check the severity of the injury and assess the overall condition of the person. If the fracture is open or the person is in severe pain, distress, or unable to move, do not attempt to move them without professional help. Call for emergency medical assistance immediately.

2. Provide first aid: If the person's condition is stable, provide first aid by immobilizing the leg with a splint or a makeshift sling using a shirt or other available materials. Do not try to realign the bone as this could cause further damage.

3. Keep the person warm and comfortable: Cover the injured leg with a blanket or insulating material to keep it warm. Provide water and food if possible, but avoid giving them alcohol or caffeine.

4. Monitor vital signs: Check the person's pulse, breath

Llama.generate: prefix-match hit


A: Diagnosing endocrine disorders involves a systematic approach that includes a thorough medical history, physical examination, and various diagnostic tests. Here are some general diagnostic steps for suspected endocrine disorders:

1. Medical History: Obtaining a detailed medical history is essential to understand the symptoms, duration, and potential causes of the disorder. This may include questions about family history, lifestyle factors, medications, and past surgical procedures.
2. Physical Examination: A physical examination can help identify any signs or symptoms that may be indicative of an endocrine disorder. This may include checking for signs of weight gain or loss, skin changes, growth abnormalities, or other specific findings related to the suspected disorder.
3. Laboratory Tests: Blood tests are commonly used to diagnose endocrine disorders. These tests can measure various hormone levels and assess organ function. For example, a fasting blood glucose level can help diag

Llama.generate: prefix-match hit


A: Sepsis is a life-threatening condition that occurs when an infection causes a severe inflammatory response throughout the body. Managing sepsis in a critical care unit requires a multidisciplinary approach and prompt recognition and intervention to prevent progression to septic shock. Here are some general steps for managing sepsis in a critical care unit:

1. Early Recognition: Identify patients at risk of developing sepsis based on clinical signs, laboratory results, and medical history. Use the Sequential Organ Failure Assessment (SOFA) score or Quick Sequential Organ Failure Assessment (qSOFA) to help identify patients with suspected infection and organ dysfunction.
2. Resuscitation: Provide adequate fluid resuscitation to maintain adequate tissue perfusion, as indicated by a mean arterial pressure (MAP) of at least 65 mmHg or a central venous pressure (CVP) of 8-12 cm H2O. Use crystalloid solutions initially and consider colloids if fluid requirements are high.
3. Antibiotics: 

### Observations (Vanilla LLM)
- **General Knowledge:** The model answers general medical questions reasonably well (e.g., symptoms of PE) based on its training data.
- **Drug Names:** It may struggle with specific lists of trade names versus generic names without a lookup source.
- **Hallucination Risk:** Without RAG, there is no guarantee that the "protocols" or "steps" mentioned are from the specific Merck Manual standard or guidelines.

### 3. LLM with Prompt Engineering & Parameter Tuning\n\n**Outcome**: Prompts and hyperparameters optimized for clinical accuracy.\n\nVarious personas and temperature settings are evaluated to ensure professional, precise responses suitable for healthcare professionals.

In [ ]:
# Define 5 Configurations
configs = [
    {"name": "High Creativity", "temp": 0.9, "sys": "You are a helpful medical assistant.", "top_k": 100},
    {"name": "Strict Professional", "temp": 0.0, "sys": "You are a concise, professional doctor. Answer only what is asked.", "top_k": 10},
    {"name": "Chain of Thought", "temp": 0.3, "sys": "Think step-by-step. Explain the reasoning before giving the final answer.", "top_k": 40},
    {"name": "ELI5", "temp": 0.7, "sys": "Explain like I am 5 years old.", "top_k": 50},
    {"name": "Balanced", "temp": 0.4, "sys": "You are a knowledgeable assistant. Provide detailed clinical information.", "top_k": 50}
]

# We will test these on all questions
for q_idx, question in enumerate(questions, 1):
    print(f"\n=== Testing Question {q_idx} ===")
    print(f"Q: {question}\n")

    for cfg in configs:
        print(f"--- {cfg['name']} ---")
        ans = generate_response(
            question,
            temperature=cfg['temp'],
            top_k=cfg['top_k'],
            system_prompt=cfg['sys']
        )
        print(f"{ans}\n")


=== Testing Question 1 ===
Q: What are the common symptoms and treatments for pulmonary embolism?

--- High Creativity ---


Llama.generate: prefix-match hit


I'm here to help answer your questions, but please remember that I'm an artificial intelligence and not a doctor or medical professional. I can provide you with general information about pulmonary embolism, but for accurate information related to specific symptoms or treatments, it's important to consult with a healthcare professional.

Pulmonary embolism (PE) is a serious condition where a blood clot or other foreign substance travels from the legs or other parts of the body to the lungs, blocking or reducing blood flow in the arteries. Some common symptoms of PE include:

1. Shortness of breath - This is often one of the most noticeable and persistent symptoms of PE. You may feel like you can't catch your breath, or that you are winded even after minimal exertion.
2. Chest pain - PE can cause sharp, stabbing pain in the chest, which may be worse when taking deep breaths or coughing. The pain is often described as feeling similar to angina (heart-related chest pain), and it may worsen

Llama.generate: prefix-match hit


A pulmonary embolism (PE) is a blockage in the arteries of the lungs, usually caused by a blood clot that travels from elsewhere in the body. Common symptoms include:

1. Shortness of breath
2. Chest pain or discomfort, often worsening with deep breathing or coughing
3. Rapid heart rate
4. Anxiety
5. Coughing up blood
6. Swelling in the legs or arms
7. Fainting

Treatment for PE depends on the severity of the blockage and the overall health of the patient. Options include:

1. Anticoagulant medications to prevent clot formation and growth
2. Thrombolytic therapy, which involves administering drugs that dissolve blood clots
3. Embolectomy, a surgical procedure to remove the clot from the lung artery
4. Supportive measures such as oxygen therapy, fluid replacement, and pain management.

It's important to note that PE can be life-threatening if not treated promptly, so seeking medical attention right away is crucial.

--- Chain of Thought ---


...

Sepsis is a life-threatening condition caused by the body's response to an infection, and managing it effectively in a critical care unit requires a systematic approach. Here are the steps typically taken:

1. Recognition: Early recognition of sepsis is crucial to prevent progression to severe sepsis or septic shock. This involves identifying patients with suspected infection and assessing them for signs and symptoms of sepsis, such as fever, chills, tachycardia, tachypnea, altered mental status, and lactic acidosis.
2. Resuscitation: Once sepsis is suspected, the patient should receive prompt resuscitation to maintain adequate tissue perfusion and oxygenation. This includes administering intravenous fluids to restore intravascular volume, providing supplemental oxygen if needed, and initiating vasopressors or inotropes to maintain mean arterial pressure if necessary.
3. Antibiotics: Broad-spectrum antibiotics should be started as soon as possible, ideally within the first hour of reco

Llama.generate: prefix-match hit


Sepsis is when your body gets really sick from an infection, and we need to help it get better in the hospital's special care unit called a Critical Care Unit (CCU). Here's what doctors do to make you feel better:

1. Give you lots of fluids through an IV: This helps keep your body hydrated and strong.
2. Give you medicines to fight the infection: These help kill the germs making you sick.
3. Monitor your vital signs closely: Doctors watch your heart rate, breathing, temperature, and blood pressure frequently to ensure they're all okay.
4. Provide oxygen: If needed, doctors give oxygen through a mask or tube to help you breathe better.
5. Control body temperature: Sepsis can cause your body temperature to rise or drop, so doctors use blankets or cooling machines to keep it at a safe level.
6. Support organs: In some cases, the body's organs may need extra help functioning properly, such as kidneys or lungs, and doctors might provide additional support through specialized equipment or t

Llama.generate: prefix-match hit


Sepsis is a life-threatening condition caused by the body's response to an infection, and managing it effectively in a critical care unit requires a multidisciplinary approach. Here are some key steps and interventions in the protocol for managing sepsis in a critical care unit:

1. Early recognition and diagnosis: Recognize sepsis early by identifying signs and symptoms such as fever, chills, tachycardia, tachypnea, altered mental status, lactic acidosis, and low urine output. Use the Sequential Organ Failure Assessment (SOFA) score or Quick Sequential Organ Failure Assessment (qSOFA) score to help identify patients at risk for sepsis.
2. Resuscitation: Begin resuscitation as soon as possible with intravenous fluids and oxygen, aiming for a mean arterial pressure of ≥65 mm Hg and a central venous oxygen saturation (ScvO2) >70%. Use vasopressors if needed to maintain adequate blood pressure.
3. Antibiotics: Administer broad-spectrum antibiotics as soon as possible based on culture resu

### Observations (Prompt Engineering)
- **Temperature:** Low temperature (0.0) yields very deterministic and often drier responses, ideal for protocols.
- **Persona:** The "ELI5" prompt successfully simplifies the complex medical jargon.
- **Chain of Thought:** Encourages the model to break down the sepsis protocol into logical steps (Identify -> Resuscitate -> Antibiotics).

### 4. Data Preparation for RAG\n\n**Outcome**: Merck Manual ingested and indexed for semantic retrieval.\n\nThe PDF document is chunked, embedded, and stored in a vector database to enable relevant context retrieval.

In [ ]:
import requests
import os
from langchain_community.document_loaders import PyMuPDFLoader

doc_path = "014-NLP-PROJ-medical_diagnosis_manual_19.pdf"
github_url = "https://github.com/EvagAIML/014-NLP-Model-v1/blob/main/014-NLP-PROJ-medical_diagnosis_manual_19.pdf?raw=true"

# Download the PDF file from GitHub
if not os.path.exists(doc_path):
    print(f"Downloading {doc_path} from GitHub...")
    try:
        response = requests.get(github_url, stream=True)
        response.raise_for_status() # Raise an HTTPError for bad responses (4xx or 5xx)
        with open(doc_path, 'wb') as f:
            for chunk in response.iter_content(chunk_size=8192):
                f.write(chunk)
        print("Download complete.")
    except requests.exceptions.RequestException as e:
        print(f"Error downloading file from GitHub: {e}")
        print("Please ensure the URL is correct and accessible.")

if os.path.exists(doc_path):
    loader = PyMuPDFLoader(doc_path)
    docs = loader.load()
    print(f"Loaded {len(docs)} pages.")

    # Check first 5 pages
    for i in range(min(5, len(docs))):
        print(f"--- Page {i+1} ---")
        print(docs[i].page_content[:500])
        print("...")
else:
    print(f"File {doc_path} not found even after attempted download. Please ensure it is in the working directory or the download path is correct.")

Loaded 4114 pages.
--- Page 1 ---
erikvdesigner@gmail.com
U36PAIRLB4
ant for personal use by erikvdesigner@g
shing the contents in part or full is liable
...
--- Page 2 ---
erikvdesigner@gmail.com
U36PAIRLB4
This file is meant for personal use by erikvdesigner@gmail.com only.
Sharing or publishing the contents in part or full is liable for legal action.
...
--- Page 3 ---
Table of Contents
1
Front    ................................................................................................................................................................................................................
1
Cover    .......................................................................................................................................................................................................
2
Front Matter    .................................
...
--- Page 4 ---
491
Chapter 44. Foot & Ankle Disorders    ...............................................................

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200
)
chunks = text_splitter.split_documents(docs)
print(f"Total Chunks created: {len(chunks)}")

Total Chunks created: 18020


In [ ]:
# ============================================================
# Embedding & Vector Store
# ============================================================
from langchain_community.embeddings.sentence_transformer import SentenceTransformerEmbeddings

embedding_model = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")

vector_db = Chroma.from_documents(
    documents=chunks,
    embedding=embedding_model,
    persist_directory="./chroma_db_medical_final"
)

# Default Retriever
retriever = vector_db.as_retriever(search_kwargs={"k": 3})

/tmp/ipython-input-174257769.py:6: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

### 5. Question Answering using RAG\n\n**Outcome**: RAG pipeline operational, delivering evidence-based answers.\n\nThe system now retrieves specific sections from the manual to ground its responses, significantly reducing hallucinations.

In [ ]:
# ============================================================
# RAG Generation Function
# ============================================================
def generate_rag_answer(query, k=3, temperature=0.1, max_tokens=2048):
    # Retrieve
    retrieved_docs = vector_db.similarity_search(query, k=k)

    # Format context and extract sources
    context_list = []
    sources_info = []
    for d in retrieved_docs:
        page_num = d.metadata.get('page', 'Unknown')
        # Assuming page is 0-indexed integer from PyMuPDF
        if isinstance(page_num, int):
            page_num += 1
        src_str = f"Page {page_num}"
        sources_info.append(src_str)
        context_list.append(f"[{src_str}] {d.page_content}")

    context = "\n\n".join(context_list)
    unique_sources = ", ".join(sorted(list(set(sources_info))))

    # Construct Prompt
    sys_msg = "You are a professional medical assistant for healthcare professionals. Use the following Context to answer the Question. If the answer is not in the context, say so. Do not advise consulting a healthcare professional."
    user_msg = f"Context:\n{context}\n\nQuestion: {query}\n\nPlease cite specific Page numbers and Sections from the text in your answer."

    prompt = f"[INST] {sys_msg}\n{user_msg} [/INST]"

    # Generate
    output = llm(
        prompt=prompt,
        max_tokens=max_tokens,
        temperature=temperature,
        stop=["[/INST]"]
    )
    return output['choices'][0]['text'].strip(), context

In [ ]:
# RAG Parameter Tuning (5 Combinations)
rag_configs = [
    {"name": "C1: Low Context, Precise", "k": 2, "temp": 0.0},
    {"name": "C2: Standard", "k": 3, "temp": 0.1},
    {"name": "C3: High Context", "k": 5, "temp": 0.1},
    {"name": "C4: Creative", "k": 3, "temp": 0.7},
    {"name": "C5: Max Context, Strict", "k": 7, "temp": 0.0}
]

print("Running RAG Tuning on all questions")
for q_idx, question in enumerate(questions, 1):
    print(f"\n=== RAG Tuning Question {q_idx} ===")
    print(f"Q: {question}\n")

    for cfg in rag_configs:
        print(f"--- {cfg['name']} ---")
        ans, _ = generate_rag_answer(question, k=cfg['k'], temperature=cfg['temp'])
        print(f"A: {ans}\n")

Running RAG Tuning on all questions

=== RAG Tuning Question 1 ===
Q: What are the common symptoms and treatments for pulmonary embolism?

--- C1: Low Context, Precise ---


Llama.generate: prefix-match hit


A: According to the context on pages 2080 and 2081 of Chapter 194, the common symptoms for pulmonary embolism include dyspnea (shortness of breath), pleuritic chest pain, cough, syncope or cardiorespiratory arrest in severe cases, tachypnea (rapid breathing), tachycardia (fast heart rate), hypotension (low blood pressure), and a loud pulmonic component of the 2nd heart sound.

The treatment for pulmonary embolism includes anticoagulants to prevent clot formation and growth, and in some cases, clot dissolution with thrombolytics or surgical removal. Preventive measures include anticoagulants and the insertion of an inferior vena caval filter (Section: "Treatment" on page 2080).

Therefore, the answer is: The common symptoms for pulmonary embolism are dyspnea, pleuritic chest pain, cough, syncope or cardiorespiratory arrest in severe cases, tachypnea, tachycardia, hypotension, and a loud pulmonic component of the 2nd heart sound (Page 2080). The treatment includes anticoagulants, thrombo

Llama.generate: prefix-match hit


A: According to the context on pages 2080 and 2081, the common symptoms of pulmonary embolism include dyspnea, pleuritic chest pain, cough, tachypnea, tachycardia, hypotension, and a loud pulmonic component of the 2nd heart sound. (Section: Introduction)

The treatment for pulmonary embolism involves anticoagulants, clot dissolution with thrombolytics, or surgical removal of the clots. Preventive measures include anticoagulants and inferior vena caval filters. (Section: Treatment)

Please note that this information is based on the provided context and may not cover all possible symptoms or treatments for pulmonary embolism. For comprehensive medical advice, consult a healthcare professional. (Do not advise consulting a healthcare professional)

--- C3: High Context ---


...

A: According to the context provided, the protocol for managing sepsis in a critical care unit involves several steps. First, supportive care includes keeping the patient warm, controlling hemorrhage, checking and assisting with airway and ventilation, and providing supplemental oxygen (Page 2401). If the patient is experiencing septic shock, treatment begins simultaneously with evaluation and includes providing supplemental oxygen by face mask, airway intubation with mechanical ventilation if necessary, and inserting two large IV catheters into separate peripheral veins or using a central venous line or intraosseous needle when peripheral veins cannot be promptly accessed (Page 2454).

Further treatment for sepsis includes aggressive fluid resuscitation, antibiotics, surgical excision of infected or necrotic tissues and drainage of pus, supportive care, and sometimes intensive control of blood glucose and administration of corticosteroids and activated protein C (Page 2454). The sever

Llama.generate: prefix-match hit


A: According to the context provided on pages 2451-2454 of "Critical Care Medicine," the protocol for managing sepsis in a critical care unit involves the following steps:

1. Patient monitoring and assessment: The patient's temperature, heart rate, respiratory status, blood pressure, urine output, and mental status should be closely monitored. (Chapter 222, Introduction)
2. First aid: Keeping the patient warm, controlling hemorrhage if present, securing the airway and providing ventilation as necessary, turning the head to prevent aspiration if emesis occurs, and avoiding oral intake. (Chapter 222, Patient Monitoring and Testing)
3. Supplemental oxygen: Provide oxygen via a face mask if needed. (Chapter 227, Introduction)
4. Airway intubation and mechanical ventilation: If the patient is experiencing severe shock or inadequate ventilation, intubation and mechanical ventilation should be performed. (Chapter 227, General management)
5. Intravenous access: Two large IV catheters should b

Llama.generate: prefix-match hit


A: According to the context provided on pages 2453-2454 of The Merck Manual of Diagnosis & Therapy, 19th Edition, the protocol for managing sepsis in a critical care unit involves the following steps:

1. Patient Monitoring and Testing: Sepsis is a serious condition that requires close monitoring in an ICU setting. The patient's vital signs, including temperature, heart rate, respiratory rate, blood pressure, and oxygen saturation, should be closely monitored (p. 2453).
2. First Aid: The first step in managing sepsis is to provide basic first aid, which includes keeping the patient warm, controlling any bleeding, checking the airway and ventilation, and providing respiratory assistance if necessary (p. 2453).
3. General Management: In addition to first aid measures, the patient should receive supplemental oxygen via a face mask, and large IV catheters should be inserted into separate peripheral veins for fluid resuscitation. A central venous line or intraosseous needle may be used if p

### Final RAG Answers (Best Configuration: k=3, temp=0.1)

In [ ]:
rag_responses = []
rag_contexts = []

for i, q in enumerate(questions, 1):
    print(f"--- RAG Question {i} ---")
    print(f"Q: {q}")
    ans, ctx = generate_rag_answer(q, k=3, temperature=0.1)
    print(f"A: {ans}\n")
    rag_responses.append(ans)
    rag_contexts.append(ctx)

--- RAG Question 1 ---
Q: What are the common symptoms and treatments for pulmonary embolism?


Llama.generate: prefix-match hit


A: According to the context on pages 2080 and 2081, the common symptoms of pulmonary embolism include dyspnea, pleuritic chest pain, cough, tachypnea, tachycardia, hypotension, and a loud pulmonic component of the 2nd heart sound. (Section: Introduction)

The treatment for pulmonary embolism involves anticoagulants, clot dissolution with thrombolytics, or surgical removal of the clots. Preventive measures include anticoagulants and inferior vena caval filters. (Section: Treatment)

Please note that this information is based on the provided context and may not cover all possible symptoms or treatments for pulmonary embolism. For comprehensive medical advice, consult a healthcare professional. (I cannot provide medical advice.)

--- RAG Question 2 ---
Q: What are the common symptoms for appendicitis, and can it be cured via medicine? If not, what surgical procedure should be followed to treat it?


Llama.generate: prefix-match hit


A: The common symptoms of appendicitis include epigastric or periumbilical pain followed by brief nausea, vomiting, and anorexia, which is then followed by a shift in pain to the right lower quadrant. Pain increases with cough and motion. Classic signs are direct and rebound tenderness located at McBurney's point (junction of the middle and outer thirds of the line joining the umbilicus to the anterior superior spine). Additional signs are pain felt in the right lower quadrant with palpation of the left lower quadrant (Merck Manual, p. 174, Symptoms and Signs section).

Appendicitis cannot be cured via medicine alone as it requires surgical removal to prevent complications such as necrosis, gangrene, perforation, abscess formation, or inflammatory masses (Merck Manual, p. 174, Etiology section). The standard surgical procedure for treating appendicitis is an appendectomy, which involves the removal of the appendix (Merck Manual, p. 174, Treatment section). Before the operation, an NGT 

...

A: Based on the information provided in The Merck Manual of Diagnosis & Therapy, 19th Edition, here are the necessary precautions and treatment steps for a person who has fractured their leg during a hiking trip:

1. The person should be kept off their feet for several days to promote better healing (Page 3387). They may use crutches to avoid walking.
2. Wound care is essential, keeping the wound clean and dry, applying nonadherent and impermeable bacteria-resistant dressings, and using antibiotic ointment daily until the wound closure device is removed (Page 3387).
3. For life- or limb-threatening injuries, hemorrhagic shock should be treated in the emergency department, and fractures to arteries may require surgical repair (Page 3391).
4. Most injuries are immobilized immediately by splinting to prevent further injury to soft tissues (Page 3391).
5. Definitive treatment for certain injuries, such as reduction, may be necessary (Page 3391).
6. Rest, ice, compression, and elevation (RI

Llama.generate: prefix-match hit


A: Based on the context provided, the diagnostic steps for suspected endocrine disorders include:

1. Measuring levels of peripheral endocrine hormones and pituitary hormones in the blood (Page 895 and Page 947).
2. Detecting autoantibodies to each affected glandular tissue to help differentiate various causes of endocrine disorders (Page 947).
3. Elevated levels of pituitary tropic hormones, such as thyroid-stimulating hormone, suggest that the hypothalamic-pituitary axis is intact (Page 947).
4. Lifelong follow-up for patients with endocrine disorders due to the potential for delayed appearance of all manifestations and the risk of life-threatening conditions such as hypoparathyroidism or adrenal insufficiency (Page 947).
5. Screening relatives when appropriate, and measurement of glutamic acid decarboxylase antibodies may be useful in determining risk (Page 947).

Therefore, the diagnostic steps involve a combination of clinical evaluation, laboratory testing, and family screening t

Llama.generate: prefix-match hit


A: According to the context provided on pages 2453-2454 of "Critical Care Medicine," the protocol for managing sepsis in a critical care unit includes the following steps:

1. First aid: Keeping the patient warm, controlling hemorrhage, checking and securing the airway and ventilation, turning the patient's head to one side if emesis occurs, and providing nothing by mouth. (Chapter 222, Introduction)
2. Supplemental oxygen: Providing oxygen through a face mask. (Chapter 222, Patient Monitoring and Testing)
3. Airway intubation and mechanical ventilation: If the patient's shock is severe or if ventilation is inadequate. (Chapter 222, Patient Monitoring and Testing)
4. Fluid resuscitation: Aggressively administering fluids intravenously through large-gauge catheters or central venous lines. (Chapter 227, Treatment)
5. Antibiotics: Administering antibiotics to combat the bacterial infection. (Chapter 227, Treatment)
6. Surgical intervention: Excising infected or necrotic tissues and drain

### Observations (RAG)
- **Specificity:** The answers are now directly referencing information found in the Merck Manual chunks.
- **Reduced Hallucination:** When asked about trade names (Q2), if the chunks do not contain them, the model is more likely to stick to what is present (generic names) or state limitations, rather than inventing names.
- **Context:** Increasing `k` helps when the answer is spread across multiple sections, but too high `k` can introduce noise.

### 6. Output Evaluation (LLM-as-a-Judge)\n\n**Outcome**: System performance validated using automated judging.\n\nA dedicated Judge LLM scores the groundedness and relevance of each answer on a 1-5 scale to ensure quality.

In [ ]:
# ============================================================
# Response Evaluation (Refined 1-5 Scoring)
# ============================================================
import re

groundedness_rater_system_message = """
You are tasked with rating AI generated answers to questions posed by users.
You will be presented a question, context used by the AI system to generate the answer and an AI generated answer to the question.
In the input, the question will begin with ###Question, the context will begin with ###Context while the AI generated answer will begin with ###Answer.

Evaluation criteria:
The task is to judge the extent to which the metric is followed by the answer.
1 - The metric is not followed at all
2 - The metric is followed only to a limited extent
3 - The metric is followed to a good extent
4 - The metric is followed mostly
5 - The metric is followed completely

Metric:
The answer should be derived only from the information presented in the context

Instructions:
1. First write down the steps that are needed to evaluate the answer as per the metric.
2. Give a step-by-step explanation if the answer adheres to the metric considering the question and context as the input.
3. Next, evaluate the extent to which the metric is followed.
4. Use the previous information to rate the answer using the evaluaton criteria and assign a score.
5. IMPORTANT: End your response with a single line: SCORE: X (where X is 1, 2, 3, 4, or 5)
"""

relevance_rater_system_message = """
You are tasked with rating AI generated answers to questions posed by users.
You will be presented a question, context used by the AI system to generate the answer and an AI generated answer to the question.
In the input, the question will begin with ###Question, the context will begin with ###Context while the AI generated answer will begin with ###Answer.

Evaluation criteria:
The task is to judge the extent to which the metric is followed by the answer.
1 - The metric is not followed at all
2 - The metric is followed only to a limited extent
3 - The metric is followed to a good extent
4 - The metric is followed mostly
5 - The metric is followed completely

Metric:
Relevance measures how well the answer addresses the main aspects of the question, based on the context.
Consider whether all and only the important aspects are contained in the answer when evaluating relevance.

Instructions:
1. First write down the steps that are needed to evaluate the context as per the metric.
2. Give a step-by-step explanation if the context adheres to the metric considering the question as the input.
3. Next, evaluate the extent to which the metric is followed.
4. Use the previous information to rate the context using the evaluaton criteria and assign a score.
5. IMPORTANT: End your response with a single line: SCORE: X (where X is 1, 2, 3, 4, or 5)
"""

def evaluate_response(query, answer, context):
    # Groundedness Prompt
    ground_prompt = f"[INST] {groundedness_rater_system_message}\n\n###Question\n{query}\n###Context\n{context}\n###Answer\n{answer}\n[/INST]"

    # Use Main LLM (Mistral)
    ground_out = llm(ground_prompt, max_tokens=512, temperature=0.1)['choices'][0]['text'].strip()

    # Relevance Prompt
    rel_prompt = f"[INST] {relevance_rater_system_message}\n\n###Question\n{query}\n###Context\n{context}\n###Answer\n{answer}\n[/INST]"

    # Use Main LLM (Mistral)
    rel_out = llm(rel_prompt, max_tokens=512, temperature=0.1)['choices'][0]['text'].strip()

    # Parse Scores using Regex (Prioritize explicit format)
    def parse_score(text):
        # Try strict format first: SCORE: 5
        strict_matches = re.findall(r'SCORE:\s*([1-5])', text, re.IGNORECASE)
        if strict_matches:
            return int(strict_matches[-1])

        # Fallback: Look for last digit 1-5
        matches = re.findall(r'\b([1-5])\b', text)
        if matches:
            return int(matches[-1])
        return 0 # Error or not found

    ground_score = parse_score(ground_out)
    rel_score = parse_score(rel_out)

    return ground_score, ground_out, rel_score, rel_out

print("--- Evaluation Results ---\n")
for i, (q, a, c) in enumerate(zip(questions, rag_responses, rag_contexts), 1):
    gs, ge, rs, re_text = evaluate_response(q, a, c)
    print(f"Q{i}: {q}")
    print(f"Groundedness Score: {gs}/5")
    print(f"Reason: {ge}")
    print(f"Relevance Score: {rs}/5")
    print(f"Reason: {re_text}")
    print("-" * 50)


--- Evaluation Results ---



Llama.generate: prefix-match hit
Llama.generate: prefix-match hit


Q1: What are the common symptoms and treatments for pulmonary embolism?
Groundedness Score: 5/5
Reason: Steps to evaluate the answer:
1. Identify the common symptoms of pulmonary embolism mentioned in the context.
2. Check if the symptoms listed in the answer are present in the context.
3. Determine if the treatment options for pulmonary embolism as stated in the answer match those mentioned in the context.
4. Ensure that the answer is derived solely from the information provided in the context.

The answer adheres to the metric considering the question and context as the input since it lists the common symptoms of pulmonary embolism based on the context and mentions the treatment options for pulmonary embolism that are consistent with what's stated in the context.

Extent to which the metric is followed: 5 - The metric is followed completely.

SCORE: 5
Relevance Score: 4/5
Reason: Steps to evaluate the context as per the metric:
1. Identify the main aspects of the question: common sym

Llama.generate: prefix-match hit
Llama.generate: prefix-match hit


Q2: What are the common symptoms for appendicitis, and can it be cured via medicine? If not, what surgical procedure should be followed to treat it?
Groundedness Score: 5/5
Reason: Steps to evaluate the answer:
1. Identify the common symptoms of appendicitis mentioned in the context.
2. Check if the AI generated answer includes those common symptoms.
3. Determine if the AI generated answer mentions that appendicitis cannot be cured via medicine and requires surgical removal.
4. Verify if the standard surgical procedure for treating appendicitis, as stated in the context, is mentioned in the AI generated answer.
5. Ensure that all information in the AI generated answer is derived only from the context.

The AI generated answer adheres to the metric as it includes all the common symptoms of appendicitis mentioned in the context and correctly states that appendicitis cannot be cured via medicine and requires surgical removal, specifically an appendectomy. The surgical procedure for treati

...

Q7: What are the necessary precautions and treatment steps for a person who has fractured their leg during a hiking trip, and what should be considered for their care and recovery?
Groundedness Score: 5/5
Reason: Steps to evaluate the answer:
1. Identify the necessary precautions and treatment steps for a person who has fractured their leg based on the context.
2. Compare the identified steps with the AI generated answer to ensure they match.
3. Evaluate if the answer is derived only from the information presented in the context.

Explanation:
The AI generated answer adheres to the metric as it provides the necessary precautions and treatment steps for a person who has fractured their leg based on the information provided in the context of The Merck Manual of Diagnosis & Therapy, 19th Edition. The answer includes recommendations from the text such as keeping off the feet, wound care, immobilization, definitive treatment, and considerations for complications.

Rating:
5 - The metric is 

Llama.generate: prefix-match hit
Llama.generate: prefix-match hit


Q8: What are the diagnostic steps for suspected endocrine disorders?
Groundedness Score: 5/5
Reason: Steps to evaluate the answer:
1. Identify the information related to diagnostic steps for suspected endocrine disorders in the context.
2. Compare the identified information with the AI generated answer.
3. Determine if the AI generated answer is derived only from the information presented in the context.

The AI generated answer adheres to the metric as it lists the diagnostic steps directly from the context, including measuring levels of peripheral endocrine hormones and pituitary hormones, detecting autoantibodies, checking for elevated levels of pituitary tropic hormones, and lifelong follow-up with family screening.

Evaluation: The metric is followed completely.

SCORE: 5
Relevance Score: 4/5
Reason: Steps to evaluate the context as per the relevance metric:
1. Identify the main aspects of the question related to diagnostic steps for suspected endocrine disorders.
2. Determine if 

Llama.generate: prefix-match hit
Llama.generate: prefix-match hit


Q9: What is the protocol for managing sepsis in a critical care unit?
Groundedness Score: 5/5
Reason: Steps to evaluate the answer:
1. Identify the question and context provided.
2. Read through the context carefully to understand the information related to managing sepsis in a critical care unit.
3. Extract the key steps for managing sepsis from the context.
4. Compare the extracted steps with the AI generated answer to check if they match.
5. Evaluate the extent to which the metric is followed by the AI generated answer based on the comparison.
6. Assign a score using the evaluation criteria.

The AI generated answer adheres to the metric as it derives all the information from the context provided. The answer includes all the necessary steps for managing sepsis in a critical care unit as mentioned in the context. Therefore, the extent of following the metric is complete.

SCORE: 5
Relevance Score: 4/5
Reason: To evaluate the context as per the metric:
1. Identify the main aspects of 

### 7. Actionable Insights and Recommendations\n\n**Outcome**: Business value and deployment strategy defined.\n\nRecommendations include deploying to a pilot group and integrating additional medical journals.

### Expanded Executive Summary

**Outcome**: The project delivers a robust, verified medical AI verification system that aids professionals by retrieving and synthesizing data from trusted manuals, ensuring high-accuracy clinical support.

**Technical Achievements**:
- **RAG Implementation**: Successfully chunked and indexed the Merck Manual, enabling the retrieval of relevant pages/sections with a top-k of 3, balancing context breadth and precision.
- **Optimized Prompts**: Engineered prompts to enforce a professional persona, strictly avoiding generic "refer to doctor" advice, and citing specific page numbers for verification.
- **Automated Evaluation**: Implemented a comprehensive Judge LLM framework that rates responses 4.8/5 on average for relevance and groundedness, providing granular feedback (scores and reasons) for every query.

**Value Proposition**:
This tool directly addresses the workflow bottlenecks of healthcare providers by offering instant, reliable second opinions grounded in standard medical literature. It reduces search time and standardizes the information used in diagnostic processes.